In [3]:
! pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 446.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227139 sha256=883c197ba1ce0f601c3eb371f226f166b9f1ab5dc1b84e1fb8aa490e9d1447dd
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import fasttext
from sklearn.model_selection import GridSearchCV
import pandas as pd

### Fast Text

In [14]:
train_df = '../content/drive/MyDrive/project_3/Data/cleaned_combined_train_df'
valid_df = '../content/drive/MyDrive/project_3/Data/cleaned_combined_valid_df'
test_df = '../content/drive/MyDrive/project_3/Data/cleaned_combined_test_df'

In [15]:
model = fasttext.train_supervised(input= train_df , autotuneValidationFile= valid_df)

In [16]:
score_test = model.test(test_df)
score_train = model.test(train_df)

In [17]:
def return_f1_score(score):

  dict_scores = {
    'precision': score[1],
    'recall': score[2],
    'F1': 2 * ((score[1]* score[2]) /(score[1] + score[2]))
  }

  return dict_scores

In [18]:
# Test score
return_f1_score(score_test)

{'precision': 0.9246231155778895,
 'recall': 0.9246231155778895,
 'F1': 0.9246231155778895}

In [19]:
#Train dataset
return_f1_score(score_train)

{'precision': 1.0, 'recall': 1.0, 'F1': 1.0}

Over fitting issue

In [20]:
df = pd.read_csv('../content/drive/MyDrive/project_3/Data/cleaned_combined_df.csv')

In [21]:
# Test
test_df_1 = df.sample(frac=0.20, random_state=42)

# Train
train_df_1 = df.drop(test_df_1.index)

train_df_1.to_csv('../content/drive/MyDrive/project_3/Data/cleaned_combined_train_df_1',columns=['subreddit_title_selftext'],index=False,header=False)
test_df_1.to_csv('../content/drive/MyDrive/project_3/Data/cleaned_combined_test_df_1',columns=['subreddit_title_selftext'],index=False,header=False)

In [22]:
train_df_1 = '../content/drive/MyDrive/project_3/Data/cleaned_combined_train_df_1'
test_df_1 = '../content/drive/MyDrive/project_3/Data/cleaned_combined_test_df_1'

In [23]:
model_2 = fasttext.train_supervised(input= train_df_1 , lr=1.0, epoch=55, wordNgrams = 2, bucket=200000, dim=100, loss='hs' , minCount=5)

score_test = model_2.test(test_df_1)
score_train = model_2.test(train_df_1)

In [24]:
# Test score
return_f1_score(score_test)

{'precision': 0.9321608040201005,
 'recall': 0.9321608040201005,
 'F1': 0.9321608040201005}

In [25]:
#Train dataset
return_f1_score(score_train)

{'precision': 1.0, 'recall': 1.0, 'F1': 1.0}

The F1 score on the training dataset shows perfect performance (1.0) for both automatic hyperparameter tuning and random search methods. However, on the testing dataset, the F1 score for automatic hyperparameter tuning is lower than that for random search by 0.01. This discrepancy may be attributed to several factors.

Firstly, the automatic hyperparameter tuning selects the best parameters based on cross-validation results on a validation dataset, while random search directly evaluates on the test dataset. If there are significant differences in the distributions between the training and test datasets, the parameters optimized for the training data might not generalize well to unseen data.

Secondly, insufficient data for training and testing could result in both precision and recall scores achieving 1.0.

Thirdly, the two subreddits  are highly distinguishable with unique bigram. As a result, there might be numerous hyperparameter combinations capable of achieving perfect accuracy or F1 scores, leading to not getting the truth hyperparameter that would also perform well on test dataset.

https://github.com/facebookresearch/fastText/tree/main/python
